# Zero to Snowflake - AISQL Functions

**Asset:** Zero to Snowflake - AISQL Functions  
**Version:** v1  
**Copyright(c):** 2025 Snowflake Inc. All rights reserved.

## AISQL関数
1. SENTIMENT()を使用してトラックの顧客レビューをポジティブ、ネガティブ、ニュートラルとしてスコア化・ラベル付け
2. AI_CLASSIFY()を使用して食品品質やサービス体験などのテーマでレビューを分類
3. EXTRACT_ANSWER()を使用してレビューテキストから特定の苦情や称賛を抽出
4. AI_SUMMARIZE_AGG()を使用してトラックブランド名ごとの顧客センチメントの簡潔な要約を生成

In [ ]:
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"tb_101_v2_aisql","version":{"major":1, "minor":1},"attributes":{"is_quickstart":0, "source":"tastybytes", "vignette": "aisql_functions"}}';

コンテキストを設定

In [ ]:
USE ROLE tb_analyst;
USE DATABASE tb_101;
USE WAREHOUSE tb_analyst_wh;

## 1. 大規模なセンチメント分析

すべてのフードトラックブランドの顧客センチメントを分析して、どのトラックが最高のパフォーマンスを発揮しているかを特定します。

**ビジネス質問:** 「各トラックブランドに対して顧客は全体的にどのように感じているか？」

### センチメントスコア範囲:
- ポジティブ: 0.5 から 1
- ニュートラル: -0.5 から 0.5
- ネガティブ: -0.5 から -1

In [ ]:
SELECT
    truck_brand_name,
    COUNT(*) AS total_reviews,
    AVG(CASE WHEN sentiment >= 0.5 THEN sentiment END) AS avg_positive_score,
    AVG(CASE WHEN sentiment BETWEEN -0.5 AND 0.5 THEN sentiment END) AS avg_neutral_score,
    AVG(CASE WHEN sentiment <= -0.5 THEN sentiment END) AS avg_negative_score
FROM (
    SELECT
        truck_brand_name,
        SNOWFLAKE.CORTEX.SENTIMENT (review) AS sentiment
    FROM harmonized.truck_reviews_v
    WHERE
        language ILIKE '%en%'
        AND review IS NOT NULL
    LIMIT 10000
)
GROUP BY
    truck_brand_name
ORDER BY total_reviews DESC;

**主要なインサイト:**

Cortex Playgroundで一度に一つずつレビューを分析していたものから、数千件を体系的に処理することへの移行に注目してください。SENTIMENT()関数はすべてのレビューを自動的にスコア化し、ポジティブ、ネガティブ、ニュートラルに分類し、即座に車両群全体の顧客満足度指標を提供しました。

## 2. 顧客フィードバックの分類

次に、すべてのレビューを分類して、顧客がサービスのどの側面について最も多く話しているかを理解しましょう。

**ビジネス質問:** 「顧客は主に何についてコメントしているか - 食品品質、サービス、配達体験？」

AI_CLASSIFY()関数を使用して、単純なキーワードマッチングではなく、AI理解に基づいてレビューをユーザー定義カテゴリに自動的に分類します。

In [ ]:
WITH classified_reviews AS (
  SELECT
    truck_brand_name,
    AI_CLASSIFY(
      review,
      ['Food Quality', 'Pricing', 'Service Experience', 'Staff Behavior']
    ):labels[0] AS feedback_category
  FROM
    harmonized.truck_reviews_v
  WHERE
    language ILIKE '%en%'
    AND review IS NOT NULL
    AND LENGTH(review) > 30
  LIMIT
    10000
)
SELECT
  truck_brand_name,
  feedback_category,
  COUNT(*) AS number_of_reviews
FROM
  classified_reviews
GROUP BY
  truck_brand_name,
  feedback_category
ORDER BY
  truck_brand_name,
  number_of_reviews DESC;

**主要なインサイト:**

AI_CLASSIFY()が数千のレビューを食品品質、サービス体験などのビジネス関連テーマに自動的に分類したことを観察してください。食品品質がトラックブランド全体で最も議論されるトピックであることがすぐにわかります。

## 3. 特定の運営インサイトの抽出

次に、非構造化テキストから正確な回答を得るために、EXTRACT_ANSWER()関数を活用します。

**ビジネス質問:** 「各顧客レビュー内で言及されている特定の運営上の問題やポジティブな言及は何か？」

In [ ]:
  SELECT
    truck_brand_name,
    primary_city,
    LEFT(review, 100) || '...' AS review_preview,
    SNOWFLAKE.CORTEX.EXTRACT_ANSWER(
        review,
        'What specific improvement or complaint is mentioned in this review?'
    ) AS specific_feedback
FROM 
    harmonized.truck_reviews_v
WHERE 
    language = 'en'
    AND review IS NOT NULL
    AND LENGTH(review) > 50
ORDER BY truck_brand_name, primary_city ASC
LIMIT 10000;

**主要なインサイト:**

EXTRACT_ANSWER()が長い顧客レビューから具体的で実行可能なインサイトを抽出していることに注目してください。この関数は「親切なスタッフが救いだった」や「ホットドッグは完璧に調理されている」などの具体的なフィードバックを自動的に特定します。

## 4. エグゼクティブサマリーの生成

最後に、顧客フィードバックの簡潔な要約を作成するために、AI_SUMMARIZE_AGG()関数を使用します。

**ビジネス質問:** 「各トラックブランドのキーテーマと全体的なセンチメントは何か？」

In [ ]:
SELECT
  truck_brand_name,
  AI_SUMMARIZE_AGG (review) AS review_summary
FROM
  (
    SELECT
      truck_brand_name,
      review
    FROM
      harmonized.truck_reviews_v
    LIMIT
      100
  )
GROUP BY
  truck_brand_name;

**主要なインサイト:**

AI_SUMMARIZE_AGG()関数は、長いレビューを明確なブランドレベルの要約に凝縮します。これらの要約は、反復テーマとセンチメントトレンドを強調し、決定者に各フードトラックのパフォーマンスの簡潔な概要を提供します。

---

私たちは、AI SQL関数の変革的な力を成功裏に実証し、顧客フィードバック分析を個々のレビュー処理から体系的で本格的な規模のインテリジェンスへと移行させました。

分析がより洗練されるにつれて、複雑なクエリで複数のAI関数を組み合わせることが困難になる場合があります。これはまさにSnowflake Copilotが強力な支援を提供する場面です。